**Shreya Singh MDS202434**

In [ ]:
!java -version


openjdk version "11.0.26" 2025-01-21
OpenJDK Runtime Environment (build 11.0.26+4-post-Ubuntu-1ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.26+4-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)


In [ ]:
!sudo apt-get update
!sudo apt-get install openjdk-11-jdk -y
!java -version


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,384 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,788 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,243 kB]
Get:13 https://r2u.stat.il

In [ ]:
!pip install pyspark==3.2.1 pydeequ==1.1.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 kB 13.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853627 sha256=9175b0d52b2e002c5abcf2538c2bfe75fc9dc353aef41e388fc807e0b5d2026b
  Stored in directory: /root/.cache/pip/wheels/37/eb/6f/30520493cfe2a4a78a5aa9942cf47bd5dff3221177b5775e24
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.5
    Uninstalling pyspark-3.5.5:
      Successfully uninstalled pyspark-3.5.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-spark-connec

In [ ]:
import os
os.environ["SPARK_VERSION"] = "3.2"


In [ ]:
from pyspark.sql import SparkSession
import pydeequ

spark = SparkSession.builder \
    .appName("AppName") \
    .config("spark.jars.packages", pydeequ.deequ_maven_coord) \
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord) \
    .config("spark.jars", "https://jdbc.postgresql.org/download/postgresql-42.6.0.jar") \
    .getOrCreate()


In [ ]:
from pydeequ.analyzers import *
from pydeequ.verification import *


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving adult.data to adult.data


In [ ]:
columns = [
    "age", "workclass", "fnlwgt", "education", "education_num", "marital_status",
    "occupation", "relationship", "race", "sex", "capital_gain", "capital_loss",
    "hours_per_week", "native_country", "income"
]

# Load the dataset using Spark
df = spark.read.csv("adult.data", header=False, inferSchema=True)
df = df.toDF(*columns)

# Show sample
df.show(5)


+---+-----------------+--------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
|age|        workclass|  fnlwgt| education|education_num|     marital_status|        occupation|  relationship|  race|    sex|capital_gain|capital_loss|hours_per_week|native_country|income|
+---+-----------------+--------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
| 39|        State-gov| 77516.0| Bachelors|         13.0|      Never-married|      Adm-clerical| Not-in-family| White|   Male|      2174.0|         0.0|          40.0| United-States| <=50K|
| 50| Self-emp-not-inc| 83311.0| Bachelors|         13.0| Married-civ-spouse|   Exec-managerial|       Husband| White|   Male|         0.0|         0.0|          13.0| United-States| <=50K|
| 38|          Private|215646.0|   HS-grad|       

In [ ]:
from pydeequ.checks import Check, CheckLevel
from pydeequ.verification import VerificationSuite
from pydeequ.verification import VerificationResult

check = Check(spark, CheckLevel.Warning, "Adult Data Check") \
    .isComplete("age") \
    .isComplete("workclass") \
    .isComplete("education") \
    .isNonNegative("age") \
    .isContainedIn("income", ["<=50K", ">50K"])

result = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(check) \
    .run()

result_df = VerificationResult.checkResultsAsDataFrame(spark, result)
result_df.show(truncate=False)


+----------------+-----------+------------+----------------------------------------------------------------------------------------------------------------------+-----------------+----------------------------------------------------+
|check           |check_level|check_status|constraint                                                                                                            |constraint_status|constraint_message                                  |
+----------------+-----------+------------+----------------------------------------------------------------------------------------------------------------------+-----------------+----------------------------------------------------+
|Adult Data Check|Warning    |Warning     |CompletenessConstraint(Completeness(age,None))                                                                        |Success          |                                                    |
|Adult Data Check|Warning    |Warning     |CompletenessConstrain

In [ ]:
from pydeequ.analyzers import *

analysis_result = (
    AnalysisRunner(spark)
    .onData(df)
    .addAnalyzer(Completeness("age"))
    .addAnalyzer(Completeness("workclass"))
    .addAnalyzer(ApproxCountDistinct("education"))
    .addAnalyzer(Mean("hours_per_week"))
    .addAnalyzer(Compliance("AgeAbove18", "age >= 18"))
    .run()
)

analysis_result_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysis_result)
analysis_result_df.show(truncate=False)


+------+--------------+-------------------+------------------+
|entity|instance      |name               |value             |
+------+--------------+-------------------+------------------+
|Column|workclass     |Completeness       |1.0               |
|Column|education     |ApproxCountDistinct|16.0              |
|Column|hours_per_week|Mean               |40.437455852092995|
|Column|AgeAbove18    |Compliance         |0.987868922944627 |
|Column|age           |Completeness       |1.0               |
+------+--------------+-------------------+------------------+



In [ ]:
analysis_result_df.toPandas().to_csv("dq_metrics.csv", index=False)
from google.colab import files
files.download("dq_metrics.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>